In [3]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
# Load the inputs
#original_tf_inplem_dir = "./tensorflow_code/"
model_dir = "uncased_L-12_H-768_A-12/"

vocab_file = model_dir + "vocab.txt"
bert_config_file = model_dir + "bert_config.json"
init_checkpoint = model_dir + "bert_model.ckpt"

input_file = "/content/drive/My Drive/Colab Notebooks/data/train-v2.0.json"
max_seq_length = 384
outside_pos = max_seq_length + 10
doc_stride = 128
max_query_length = 64
max_answer_length = 30
output_dir = "output_model"
learning_rate = 3e-5

In [3]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2019-03-13 02:50:39--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.206.128, 2a00:1450:400c:c04::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.206.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip.2’

uncased_L-12_H-768_ 100%[===================>] 388.84M   157MB/s    in 2.5s    

2019-03-13 02:50:41 (157 MB/s) - ‘uncased_L-12_H-768_A-12.zip.2’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
replace uncased_L-12_H-768_A-12/bert_model.ckpt.meta? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating

# Try using normal run squad script


In [4]:
!pip install bert-tensorflow

In [0]:
from bert import modeling
from bert import tokenization
from bert import run_squad
from bert import optimization
import random
import os
import collections
import six
import json

In [0]:
!cp /content/drive/My\ Drive/Colab\ Notebooks/data/output_model.zip /content
!unzip output_model.zip

Archive:  output_model.zip
   creating: output_model/
  inflating: output_model/model.ckpt-500.meta  
  inflating: output_model/model.ckpt-0.index  
  inflating: output_model/graph.pbtxt  
  inflating: output_model/model.ckpt-760.meta  
  inflating: output_model/checkpoint  
  inflating: output_model/model.ckpt-500.data-00000-of-00001  
  inflating: output_model/events.out.tfevents.1551996110.a75c072e79ed  
  inflating: output_model/model.ckpt-0.meta  
  inflating: output_model/model.ckpt-760.index  
  inflating: output_model/model.ckpt-500.index  
  inflating: output_model/model.ckpt-760.data-00000-of-00001  
  inflating: output_model/model.ckpt-0.data-00000-of-00001  
  inflating: output_model/train.tf_record  
  inflating: output_model/events.out.tfevents.1552188869.d956a7f92d9e  
  inflating: output_model/model.ckpt-750.index  
  inflating: output_model/model.ckpt-750.meta  
  inflating: output_model/model.ckpt-750.data-00000-of-00001  


In [0]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)


In [0]:
tokenizer = tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=True)


In [0]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100


In [0]:
import tensorflow as tf

#is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.estimator.RunConfig(
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    save_summary_steps = SAVE_SUMMARY_STEPS)

In [0]:
def read_squad_examples(input_file, is_training,version_2_with_negative):
  """Read a SQuAD json file into a list of SquadExample."""
  with tf.gfile.Open(input_file, "r") as reader:
    input_data = json.load(reader)["data"]

  def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
      return True
    return False

  examples = []
  for entry in input_data:
    for paragraph in entry["paragraphs"]:
      paragraph_text = paragraph["context"]
      doc_tokens = []
      char_to_word_offset = []
      prev_is_whitespace = True
      for c in paragraph_text:
        if is_whitespace(c):
          prev_is_whitespace = True
        else:
          if prev_is_whitespace:
            doc_tokens.append(c)
          else:
            doc_tokens[-1] += c
          prev_is_whitespace = False
        char_to_word_offset.append(len(doc_tokens) - 1)

      for qa in paragraph["qas"]:
        qas_id = qa["id"]
        question_text = qa["question"]
        start_position = None
        end_position = None
        orig_answer_text = None
        is_impossible = False
        if is_training:

          if version_2_with_negative:
            is_impossible = qa["is_impossible"]
          if (len(qa["answers"]) != 1) and (not is_impossible):
            raise ValueError(
                "For training, each question should have exactly 1 answer.")
          if not is_impossible:
            answer = qa["answers"][0]
            orig_answer_text = answer["text"]
            answer_offset = answer["answer_start"]
            answer_length = len(orig_answer_text)
            start_position = char_to_word_offset[answer_offset]
            end_position = char_to_word_offset[answer_offset + answer_length -
                                               1]
            # Only add answers where the text can be exactly recovered from the
            # document. If this CAN'T happen it's likely due to weird Unicode
            # stuff so we will just skip the example.
            #
            # Note that this means for training mode, every example is NOT
            # guaranteed to be preserved.
            actual_text = " ".join(
                doc_tokens[start_position:(end_position + 1)])
            cleaned_answer_text = " ".join(
                tokenization.whitespace_tokenize(orig_answer_text))
            if actual_text.find(cleaned_answer_text) == -1:
              tf.logging.warning("Could not find answer: '%s' vs. '%s'",
                                 actual_text, cleaned_answer_text)
              continue
          else:
            start_position = -1
            end_position = -1
            orig_answer_text = ""

        example = run_squad.SquadExample(
            qas_id=qas_id,
            question_text=question_text,
            doc_tokens=doc_tokens,
            orig_answer_text=orig_answer_text,
            start_position=start_position,
            end_position=end_position,
            is_impossible=is_impossible)
        examples.append(example)

  return examples


In [0]:
# Convert examples to features for training and testing
#train_file = "/content/drive/My Drive/Colab Notebooks/data/train-v2.0.json"
train_file = "/content/drive/My Drive/Colab Notebooks/data/monitoring (1).json"
#test_file = "/content/drive/My Drive/Colab Notebooks/data/dev-v2.0.json"

In [0]:
train_examples = read_squad_examples(input_file=train_file, is_training=True,version_2_with_negative=True)

# Get random sample of it because of size
#train_examples = [train_examples[random.randrange(len(train_examples))]
              #for item in range(8000)]


In [0]:
# Shuffle the training examples
rng = random.Random(12345)
rng.shuffle(train_examples)

In [0]:
def model_fn_builder(bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (start_logits, end_logits) = create_model(
        bert_config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    tvars = tf.trainable_variables()

    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      seq_length = modeling.get_shape_list(input_ids)[1]

      def compute_loss(logits, positions):
        one_hot_positions = tf.one_hot(
            positions, depth=seq_length, dtype=tf.float32)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        loss = -tf.reduce_mean(
            tf.reduce_sum(one_hot_positions * log_probs, axis=-1))
        return loss

      start_positions = features["start_positions"]
      end_positions = features["end_positions"]

      start_loss = compute_loss(start_logits, start_positions)
      end_loss = compute_loss(end_logits, end_positions)

      total_loss = (start_loss + end_loss) / 2.0

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.estimator.EstimatorSpec(
                mode=mode,
                loss=total_loss,
                train_op=train_op)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      predictions = {
          "unique_ids": unique_ids,
          "start_logits": start_logits,
          "end_logits": end_logits,
      }
      output_spec = tf.estimator.EstimatorSpec(
          mode=mode, predictions=predictions)
    else:
      raise ValueError(
          "Only TRAIN and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [0]:
#!rm -r output_model
!mkdir output_model

In [21]:
train_writer = run_squad.FeatureWriter(
        filename=os.path.join(output_dir, "train.tf_record"),
        is_training=True)
run_squad.convert_examples_to_features(
    examples=train_examples,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    doc_stride=doc_stride,
    max_query_length=max_query_length,
    is_training=True,
    output_fn=train_writer.process_feature)
train_writer.close()

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 1000000000
INFO:tensorflow:example_index: 0
INFO:tensorflow:doc_span_index: 0
INFO:tensorflow:tokens: [CLS] what is the trend of sales by household _ type [SEP] the top department for quantity between jul 2015 and jun 2017 was this . the top brand for quantity between jul 2015 and jun 2017 was this . the top commodity for quantity between jul 2015 and jun 2017 was this . the top age _ band for quantity between jul 2015 and jun 2017 was this . the top loyalty for quantity between jul 2015 and jun 2017 was this . the top household _ type for quantity between jul 2015 and jun 2017 was this . the top household _ type for quantity between jul 2015 and jun 2017 was this . . the bottom department for quantity between jul 2015 and jun 2017 was this . the bottom brand for quantity between jul 2015 and jun 2017 was this . the bottom commodity for quantity between jul 2015 and jun 2017 was this . the bottom age _ band for quantity between

In [23]:
num_train_steps = int(len(train_examples) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
#num_train_steps=1500
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
num_train_steps

21

In [0]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=init_checkpoint,
      learning_rate=learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=False,
      use_one_hot_embeddings=False)

In [25]:
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': 'output_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3b172c9be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [0]:

def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  final_hidden = model.get_sequence_output()

  final_hidden_shape = modeling.get_shape_list(final_hidden, expected_rank=3)
  batch_size = final_hidden_shape[0]
  seq_length = final_hidden_shape[1]
  hidden_size = final_hidden_shape[2]

  output_weights = tf.get_variable(
      "cls/squad/output_weights", [2, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "cls/squad/output_bias", [2], initializer=tf.zeros_initializer())

  final_hidden_matrix = tf.reshape(final_hidden,
                                   [batch_size * seq_length, hidden_size])
  logits = tf.matmul(final_hidden_matrix, output_weights, transpose_b=True)
  logits = tf.nn.bias_add(logits, output_bias)

  logits = tf.reshape(logits, [batch_size, seq_length, 2])
  logits = tf.transpose(logits, [2, 0, 1])

  unstacked_logits = tf.unstack(logits, axis=0)

  (start_logits, end_logits) = (unstacked_logits[0], unstacked_logits[1])

  return (start_logits, end_logits)

In [27]:
train_input_fn = run_squad.input_fn_builder(
        input_file=train_writer.filename,
        seq_length=max_seq_length,
        is_training=True,
        drop_remainder=False)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)


Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = end_positions, shape = (?,)
INFO:tensorflow:  name = input_ids, shape = (?, 384)
INFO:tensorflow:  name = input_mask, shape = (?, 384)
INFO:tensorflow:  name = segment_ids, shape = (?, 384)
INFO:tensorflow:  name = start_positions, shape = (?,)
INFO:tensorflow:  name = unique_ids, shape = (?,)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tens

In [0]:
!zip -r /content/drive/My\ Drive/Colab\ Notebooks/data/output_model.zip output_model

updating: output_model/ (stored 0%)
updating: output_model/model.ckpt-500.meta (deflated 92%)
updating: output_model/model.ckpt-0.index (deflated 77%)
updating: output_model/graph.pbtxt (deflated 97%)
updating: output_model/checkpoint (deflated 67%)
updating: output_model/model.ckpt-500.data-00000-of-00001 (deflated 11%)
updating: output_model/model.ckpt-0.meta (deflated 92%)
updating: output_model/model.ckpt-500.index (deflated 69%)
updating: output_model/model.ckpt-0.data-00000-of-00001 (deflated 69%)
updating: output_model/train.tf_record (deflated 78%)
updating: output_model/eval.tf_record (deflated 78%)
  adding: output_model/model.ckpt-1000.data-00000-of-00001 (deflated 11%)
  adding: output_model/model.ckpt-1000.meta (deflated 92%)
  adding: output_model/model.ckpt-1000.index (deflated 69%)


# Test out examples now

In [0]:
test_file = "/content/drive/My Drive/Colab Notebooks/data/dev-v2.0.json"

In [0]:
eval_examples = read_squad_examples(
        input_file=test_file, is_training=False,version_2_with_negative=True)
# Get random sample of it because of size
eval_examples = [eval_examples[random.randrange(len(eval_examples))]
              for item in range(1000)]

In [0]:
eval_writer = run_squad.FeatureWriter(
        filename=os.path.join(output_dir, "eval.tf_record"),
        is_training=False)
eval_features = []

def append_feature(feature):
  eval_features.append(feature)
  eval_writer.process_feature(feature)

run_squad.convert_examples_to_features(
        examples=eval_examples,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_training=False,
        output_fn=append_feature)
eval_writer.close()

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 1000000000
INFO:tensorflow:example_index: 0
INFO:tensorflow:doc_span_index: 0
INFO:tensorflow:tokens: [CLS] in what irish city can few signs remain to be seen of the hug ##uen ##ots ? [SEP] a number of hug ##uen ##ots served as mayors in dublin , cork , you ##gh ##al and waterford in the 17th and 18th centuries . numerous signs of hug ##uen ##ot presence can still be seen with names still in use , and with areas of the main towns and cities named after the people who settled there . examples include the hug ##uen ##ot district and french church street in cork city ; and d ' ol ##ier street in dublin , named after a high sheriff and one of the founders of the bank of ireland . a french church in port ##ar ##lington dates back to 169 ##6 , and was built to serve the significant new hug ##uen ##ot community in the town . at the time , they constituted the majority of the townspeople . [SEP]
INFO:tensorflow:token_to_orig_map: 19:0 

In [0]:
predict_input_fn = run_squad.input_fn_builder(
        input_file=eval_writer.filename,
        seq_length=max_seq_length,
        is_training=False,
        drop_remainder=False)

In [0]:
all_results = []
for result in estimator.predict(
  predict_input_fn, yield_single_examples=True):
  if len(all_results) % 1000 == 0:
    tf.logging.info("Processing example: %d" % (len(all_results)))
  unique_id = int(result["unique_ids"])
  start_logits = [float(x) for x in result["start_logits"].flat]
  end_logits = [float(x) for x in result["end_logits"].flat]
  all_results.append(
      run_squad.RawResult(
          unique_id=unique_id,
          start_logits=start_logits,
          end_logits=end_logits))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 384)
INFO:tensorflow:  name = input_mask, shape = (?, 384)
INFO:tensorflow:  name = segment_ids, shape = (?, 384)
INFO:tensorflow:  name = unique_ids, shape = (?,)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, 

In [0]:
def write_predictions(all_examples, all_features, all_results, n_best_size,
                      max_answer_length, do_lower_case):
  """Write final predictions to the json file and log-odds of null if needed."""
  #tf.logging.info("Writing predictions to: %s" % (output_prediction_file))
  #tf.logging.info("Writing nbest to: %s" % (output_nbest_file))

  example_index_to_features = collections.defaultdict(list)
  for feature in all_features:
    example_index_to_features[feature.example_index].append(feature)

  unique_id_to_result = {}
  for result in all_results:
    unique_id_to_result[result.unique_id] = result

  _PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
      "PrelimPrediction",
      ["feature_index", "start_index", "end_index", "start_logit", "end_logit"])

  all_predictions = collections.OrderedDict()
  all_nbest_json = collections.OrderedDict()
  scores_diff_json = collections.OrderedDict()

  for (example_index, example) in enumerate(all_examples):
    features = example_index_to_features[example_index]

    prelim_predictions = []
    # keep track of the minimum score of null start+end of position 0
    score_null = 1000000  # large and positive
    min_null_feature_index = 0  # the paragraph slice with min mull score
    null_start_logit = 0  # the start logit at the slice with min null score
    null_end_logit = 0  # the end logit at the slice with min null score
    for (feature_index, feature) in enumerate(features):
      result = unique_id_to_result[feature.unique_id]
      start_indexes = run_squad._get_best_indexes(result.start_logits, n_best_size)
      end_indexes = run_squad._get_best_indexes(result.end_logits, n_best_size)
      # if we could have irrelevant answers, get the min score of irrelevant
      if version_2_with_negative:
        feature_null_score = result.start_logits[0] + result.end_logits[0]
        if feature_null_score < score_null:
          score_null = feature_null_score
          min_null_feature_index = feature_index
          null_start_logit = result.start_logits[0]
          null_end_logit = result.end_logits[0]
      for start_index in start_indexes:
        for end_index in end_indexes:
          # We could hypothetically create invalid predictions, e.g., predict
          # that the start of the span is in the question. We throw out all
          # invalid predictions.
          if start_index >= len(feature.tokens):
            continue
          if end_index >= len(feature.tokens):
            continue
          if start_index not in feature.token_to_orig_map:
            continue
          if end_index not in feature.token_to_orig_map:
            continue
          if not feature.token_is_max_context.get(start_index, False):
            continue
          if end_index < start_index:
            continue
          length = end_index - start_index + 1
          if length > max_answer_length:
            continue
          prelim_predictions.append(
              _PrelimPrediction(
                  feature_index=feature_index,
                  start_index=start_index,
                  end_index=end_index,
                  start_logit=result.start_logits[start_index],
                  end_logit=result.end_logits[end_index]))

    if version_2_with_negative:
      prelim_predictions.append(
          _PrelimPrediction(
              feature_index=min_null_feature_index,
              start_index=0,
              end_index=0,
              start_logit=null_start_logit,
              end_logit=null_end_logit))
    prelim_predictions = sorted(
        prelim_predictions,
        key=lambda x: (x.start_logit + x.end_logit),
        reverse=True)

    _NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "NbestPrediction", ["text", "start_logit", "end_logit"])

    seen_predictions = {}
    nbest = []
    for pred in prelim_predictions:
      if len(nbest) >= n_best_size:
        break
      feature = features[pred.feature_index]
      if pred.start_index > 0:  # this is a non-null prediction
        tok_tokens = feature.tokens[pred.start_index:(pred.end_index + 1)]
        orig_doc_start = feature.token_to_orig_map[pred.start_index]
        orig_doc_end = feature.token_to_orig_map[pred.end_index]
        orig_tokens = example.doc_tokens[orig_doc_start:(orig_doc_end + 1)]
        tok_text = " ".join(tok_tokens)

        # De-tokenize WordPieces that have been split off.
        tok_text = tok_text.replace(" ##", "")
        tok_text = tok_text.replace("##", "")

        # Clean whitespace
        tok_text = tok_text.strip()
        tok_text = " ".join(tok_text.split())
        orig_text = " ".join(orig_tokens)

        final_text = get_final_text(tok_text, orig_text, do_lower_case)
        if final_text in seen_predictions:
          continue

        seen_predictions[final_text] = True
      else:
        final_text = ""
        seen_predictions[final_text] = True

      nbest.append(
          _NbestPrediction(
              text=final_text,
              start_logit=pred.start_logit,
              end_logit=pred.end_logit))

    # if we didn't inlude the empty option in the n-best, inlcude it
    if version_2_with_negative:
      if "" not in seen_predictions:
        nbest.append(
            _NbestPrediction(
                text="", start_logit=null_start_logit,
                end_logit=null_end_logit))
    # In very rare edge cases we could have no valid predictions. So we
    # just create a nonce prediction in this case to avoid failure.
    if not nbest:
      nbest.append(
          _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

    assert len(nbest) >= 1

    total_scores = []
    best_non_null_entry = None
    for entry in nbest:
      total_scores.append(entry.start_logit + entry.end_logit)
      if not best_non_null_entry:
        if entry.text:
          best_non_null_entry = entry

    probs = run_squad._compute_softmax(total_scores)

    nbest_json = []
    for (i, entry) in enumerate(nbest):
      output = collections.OrderedDict()
      output["text"] = entry.text
      output["probability"] = probs[i]
      output["start_logit"] = entry.start_logit
      output["end_logit"] = entry.end_logit
      nbest_json.append(output)

    assert len(nbest_json) >= 1

    if not version_2_with_negative:
      all_predictions[example.qas_id] = nbest_json[0]["text"]
    else:
      # predict "" iff the null score - the score of best non-null > threshold
      score_diff = score_null - best_non_null_entry.start_logit - (
          best_non_null_entry.end_logit)
      scores_diff_json[example.qas_id] = score_diff
      if score_diff > null_score_diff_threshold:
        all_predictions[example.qas_id] = ""
      else:
        all_predictions[example.qas_id] = best_non_null_entry.text

    all_nbest_json[example.qas_id] = nbest_json

  return all_predictions,all_nbest_json
 
def get_final_text(pred_text, orig_text, do_lower_case):
  """Project the tokenized prediction back to the original text."""

  # When we created the data, we kept track of the alignment between original
  # (whitespace tokenized) tokens and our WordPiece tokenized tokens. So
  # now `orig_text` contains the span of our original text corresponding to the
  # span that we predicted.
  #
  # However, `orig_text` may contain extra characters that we don't want in
  # our prediction.
  #
  # For example, let's say:
  #   pred_text = steve smith
  #   orig_text = Steve Smith's
  #
  # We don't want to return `orig_text` because it contains the extra "'s".
  #
  # We don't want to return `pred_text` because it's already been normalized
  # (the SQuAD eval script also does punctuation stripping/lower casing but
  # our tokenizer does additional normalization like stripping accent
  # characters).
  #
  # What we really want to return is "Steve Smith".
  #
  # Therefore, we have to apply a semi-complicated alignment heruistic between
  # `pred_text` and `orig_text` to get a character-to-charcter alignment. This
  # can fail in certain cases in which case we just return `orig_text`.

  def _strip_spaces(text):
    ns_chars = []
    ns_to_s_map = collections.OrderedDict()
    for (i, c) in enumerate(text):
      if c == " ":
        continue
      ns_to_s_map[len(ns_chars)] = i
      ns_chars.append(c)
    ns_text = "".join(ns_chars)
    return (ns_text, ns_to_s_map)

  # We first tokenize `orig_text`, strip whitespace from the result
  # and `pred_text`, and check if they are the same length. If they are
  # NOT the same length, the heuristic has failed. If they are the same
  # length, we assume the characters are one-to-one aligned.
  tokenizer = tokenization.BasicTokenizer(do_lower_case=do_lower_case)

  tok_text = " ".join(tokenizer.tokenize(orig_text))

  start_position = tok_text.find(pred_text)
  if start_position == -1:
    if verbose_logging:
      tf.logging.info(
          "Unable to find text: '%s' in '%s'" % (pred_text, orig_text))
    return orig_text
  end_position = start_position + len(pred_text) - 1

  (orig_ns_text, orig_ns_to_s_map) = _strip_spaces(orig_text)
  (tok_ns_text, tok_ns_to_s_map) = _strip_spaces(tok_text)

  if len(orig_ns_text) != len(tok_ns_text):
    if verbose_logging:
      tf.logging.info("Length not equal after stripping spaces: '%s' vs '%s'",
                      orig_ns_text, tok_ns_text)
    return orig_text

  # We then project the characters in `pred_text` back to `orig_text` using
  # the character-to-character alignment.
  tok_s_to_ns_map = {}
  for (i, tok_index) in six.iteritems(tok_ns_to_s_map):
    tok_s_to_ns_map[tok_index] = i

  orig_start_position = None
  if start_position in tok_s_to_ns_map:
    ns_start_position = tok_s_to_ns_map[start_position]
    if ns_start_position in orig_ns_to_s_map:
      orig_start_position = orig_ns_to_s_map[ns_start_position]

  if orig_start_position is None:
    if verbose_logging:
      tf.logging.info("Couldn't map start position")
    return orig_text

  orig_end_position = None
  if end_position in tok_s_to_ns_map:
    ns_end_position = tok_s_to_ns_map[end_position]
    if ns_end_position in orig_ns_to_s_map:
      orig_end_position = orig_ns_to_s_map[ns_end_position]

  if orig_end_position is None:
    if verbose_logging:
      tf.logging.info("Couldn't map end position")
    return orig_text

  output_text = orig_text[orig_start_position:(orig_end_position + 1)]
  return output_text


In [0]:

n_best_size=20
do_lower_case=True
verbose_logging=False
version_2_with_negative=True
null_score_diff_threshold=0.0
all_predictions,all_nbest_json = write_predictions(eval_examples, eval_features, all_results,
                  n_best_size, max_answer_length,
                  do_lower_case)

In [0]:
with open(test_file) as f:
    eval_answers = json.load(f)
#eval_answers


In [0]:
import pandas as pd
master_DF=pd.DataFrame()
for data in eval_answers['data']:
  paragraphs = data['paragraphs']
  for paragraph in paragraphs:
    qas = paragraph['qas']
    for qa in qas:
      this_id = qa['id']
      answers = qa['answers']
      for answer in answers:
        this_text = answer['text']
        this_df = pd.DataFrame(data = {'text':this_text,'id':this_id},index=[0])
        # Add to master DF
        master_DF = master_DF.append(this_df)


In [0]:
actual_DF = master_DF.drop_duplicates()
predict_text = [x[1] for x in list(all_predictions.items())]
ids = list(all_predictions)
counting_vector = 0
ids_correct = []
import re

for i in range(len(all_predictions)):
  #print(i)
  actual_text = (actual_DF[actual_DF['id']==ids[i]]['text'])
  this_predict_text = str(predict_text[i])
  for this_text in actual_text:
    #print(this_text)
    #print(this_predict_text)
    
    # Do some cleaning up
    this_text = re.sub('[^a-zA-Z0-9 ]', '', this_text)
    this_predict_text = re.sub('[^a-zA-Z0-9 ]', '', this_predict_text)

    if this_text==this_predict_text:
      counting_vector=counting_vector+1
      ids_correct.append(ids[i])
    elif re.search(this_text, this_predict_text):
      counting_vector=counting_vector+1
      ids_correct.append(ids[i])
    elif re.search(this_predict_text, this_text):
      counting_vector=counting_vector+1
      ids_correct.append(ids[i])
    else:
      print(this_text + " " + str(1))
      print(this_predict_text + " " + str(2))

Sankt Goarshausen 1
Lorelei 2
Near Sankt Goarshausen 1
Lorelei 2
10000 m2 1
5111 m2 5501435 sq ft 2
more than 10000 m2 1
5111 m2 5501435 sq ft 2
all 1
60 2
negative effect 1
High and persistent unemployment in which inequality increases has a negative 2
better 1
much higher school fees 2
produce better academic results 1
much higher school fees 2
among states in the US with larger income inequalities 1
countries with bigger income inequalities 2
using sickles to deflate one of the large domes covering two satellite dishes 1
GCSB Waihopai 2
using sickles to deflate one of the large domes 1
GCSB Waihopai 2
algebraic 1
identity 2
modern algebraic number theory 1
identity 2
infrequent rain 1
Mediterranean 2
easier credit 1
unsustainable monetary stimulation 2
to distract Montcalm 1
a massacre 2
distract Montcalm 1
a massacre 2
late 19th 1
19th century 2
A or  1
n  p  2n  2 2
deterministic 1
general numbers n 2
deterministic algorithms 1
general numbers n 2
Complexity measures 1
circuit com

In [0]:
accuracy = counting_vector/len(all_predictions)
print('Accuracy is ' + str(accuracy))

Accuracy is 0.7021943573667712


In [0]:
ids_incorrect = [i for i in ids if i not in ids_correct]
ids_incorrect

['5727f05b4b864d190016406b',
 '5acff3b377cf76001a6865fc',
 '57332b66d058e614000b5758',
 '5730042804bcaa1900d77011',
 '5ad26dbcd7d075001a4293d5',
 '571a49de4faf5e1900b8a94c',
 '5728349dff5b5019007d9f01',
 '5a6ce4fa4eec6b001a80a68b',
 '5ad26797d7d075001a429236',
 '571127a5a58dae1900cd6cc6',
 '5ad507d25b96ef001a10aa4c',
 '5ad02e5d77cf76001a686d3c',
 '5ad26dbcd7d075001a4293d6',
 '5ad21e1ad7d075001a428471',
 '5a63772268151a001a9222f4',
 '5a1c850fb4fb5d001871464e',
 '5ad24057d7d075001a428930',
 '57300f8504bcaa1900d770d1',
 '5a25d89aef59cd001a623cf8',
 '57309bfb8ab72b1400f9c5e9',
 '5a605197eae51e001ab14d0f',
 '5ad3ba81604f3c001a3fee9e',
 '572faec7b2c2fd1400568333',
 '572fffb404bcaa1900d76ff0',
 '573735e8c3c5551400e51e72',
 '5a2c07d5bfd06b001a5ae983',
 '57268882f1498d1400e8e307',
 '5a8940ba3b2508001a72a55e',
 '5ad568d35b96ef001a10ae1b',
 '5a3e42de378766001a00252a',
 '5a81f0f031013a001a334fa9',
 '5727db85ff5b5019007d96fc',
 '5ad3a27c604f3c001a3fea31',
 '5a89473c3b2508001a72a59c',
 '57273a465951

In [0]:
all_predictions

OrderedDict([('5727f05b4b864d190016406b', 'the poor'),
             ('5acff3b377cf76001a6865fc', ''),
             ('57332b66d058e614000b5758', '1915 until November 1918'),
             ('5729f39a6aef051400155150', 'granulysin'),
             ('5730042804bcaa1900d77011', ''),
             ('57106644b654c5140001f8e9', 'Paris'),
             ('5ad26dbcd7d075001a4293d5', '1724'),
             ('57264cac708984140094c1b7', 'rationing'),
             ('572fdc34a23a5019007fca94', 'Edinburgh Pentlands'),
             ('571a49de4faf5e1900b8a94c', ''),
             ('5728349dff5b5019007d9f01', ''),
             ('5a6ce4fa4eec6b001a80a68b', 'assess patient drug allergies'),
             ('5ad26797d7d075001a429236', 'conceptual definition of force'),
             ('57285ed5ff5b5019007da1b9', 'Kublai Khan'),
             ('571127a5a58dae1900cd6cc6', ''),
             ('573362b94776f41900660974', '17th century'),
             ('5ad507d25b96ef001a10aa4c', ''),
             ('5ad02e5d77cf76001a686d3c'

In [0]:
all_predictions=='5727f05b4b864d190016406b'

#Try to run for our own examples

In [0]:
input_question = ["When is the deal due","Who is the prime minister","Who is Jean Claude Juncker"]
input_context = "The British government has won what it says are crucial improvements to the Brexit divorce deal, due to be voted on by Parliament on Tuesday (UK time). That vote is the UK's last chance to secure a Brexit that avoids either a delayed or a damaging 'no deal' exit from the EU, due on March 29. Government Minister David Lidlington said new legally binding assurances from the EU, agreed in a last-gasp negotiation on Monday, ensured the UK would not end up trapped subject to EU rules and courts indefinitely – as some Brexiteers had feared. Prime Minister Theresa May said she had secured the legal changes to the divorce deal that Parliament had wanted.”Now is the time to come together to back this improved Brexit deal and deliver on the instruction of the British people,” May said, in a late-night press conference after her unscheduled trip to Strasbourg to conclude negotiations with EU Commission President Jean-Claude Juncker. Juncker warned that the new \”clarifications and guarantees” he had given on the divorce deal, in a legally binding instrument to accompany the official Withdrawal Agreement, were the final concessions the EU would allow on Brexit. ”In politics, sometimes you get a second chance,” he said. ”It’s what you do with the second chance that counts. Because there will be no third chance. No further interpretations or reassurances. If [the deal] fails, it fails tomorrow.” But within minutes of the new documents being revealed, some MPs and commentators dismissed them as minor footnotes to a deal essentially unchanged from the one agreed in November and overwhelmingly rejected by the House of Commons in January."

In [0]:
input_question = ["What is the trend of Sales by Household_Type"]
input_context = "The top Department for Quantity between Jul 2015 and Jun 2017 was this. The top Brand for Quantity between Jul 2015 and Jun 2017 was this. The top Commodity for Quantity between Jul 2015 and Jun 2017 was this. The top Age_Band for Quantity between Jul 2015 and Jun 2017 was this. The top Loyalty for Quantity between Jul 2015 and Jun 2017 was this. The top Household_Type for Quantity between Jul 2015 and Jun 2017 was this. The top Household_Type for Quantity between Jul 2015 and Jun 2017 was this.. The bottom Department for Quantity between Jul 2015 and Jun 2017 was this. The bottom Brand for Quantity between Jul 2015 and Jun 2017 was this. The bottom Commodity for Quantity between Jul 2015 and Jun 2017 was this. The bottom Age_Band for Quantity between Jul 2015 and Jun 2017 was this. The bottom Loyalty for Quantity between Jul 2015 and Jun 2017 was this. The bottom Household_Type for Quantity between Jul 2015 and Jun 2017 was this. The bottom Household_Type for Quantity between Jul 2015 and Jun 2017 was this.. The breakdown of Quantity between Jul 2015 and Jun 2017 was this for Department. The breakdown of Quantity between Jul 2015 and Jun 2017 was this for Brand. The breakdown of Quantity between Jul 2015 and Jun 2017 was this for Commodity. The breakdown of Quantity between Jul 2015 and Jun 2017 was this for Age_Band. The breakdown of Quantity between Jul 2015 and Jun 2017 was this for Loyalty. The breakdown of Quantity between Jul 2015 and Jun 2017 was this for Household_Type. The breakdown of Quantity between Jul 2015 and Jun 2017 was this. Between Jul 2015 and Jun 2017, the total Quantity trend was this for Department. Between Jul 2015 and Jun 2017, the total Quantity trend was this for Brand. Between Jul 2015 and Jun 2017, the total Quantity trend was this for Commodity. Between Jul 2015 and Jun 2017, the total Quantity trend was this for Age_Band. Between Jul 2015 and Jun 2017, the total Quantity trend was this for Loyalty. Between Jul 2015 and Jun 2017, the total Quantity trend was this for Household_Type. Between Jul 2015 and Jun 2017, the total Quantity trend was this. The top Department for Sales between Jul 2015 and Jun 2017 was this. The top Brand for Sales between Jul 2015 and Jun 2017 was this. The top Commodity for Sales between Jul 2015 and Jun 2017 was this. The top Age_Band for Sales between Jul 2015 and Jun 2017 was this. The top Loyalty for Sales between Jul 2015 and Jun 2017 was this. The top Household_Type for Sales between Jul 2015 and Jun 2017 was this. The top Household_Type for Sales between Jul 2015 and Jun 2017 was this.. The bottom Department for Sales between Jul 2015 and Jun 2017 was this. The bottom Brand for Sales between Jul 2015 and Jun 2017 was this. The bottom Commodity for Sales between Jul 2015 and Jun 2017 was this. The bottom Age_Band for Sales between Jul 2015 and Jun 2017 was this. The bottom Loyalty for Sales between Jul 2015 and Jun 2017 was this. The bottom Household_Type for Sales between Jul 2015 and Jun 2017 was this. The bottom Household_Type for Sales between Jul 2015 and Jun 2017 was this.. The breakdown of Sales between Jul 2015 and Jun 2017 was this for Department. The breakdown of Sales between Jul 2015 and Jun 2017 was this for Brand. The breakdown of Sales between Jul 2015 and Jun 2017 was this for Commodity. The breakdown of Sales between Jul 2015 and Jun 2017 was this for Age_Band. The breakdown of Sales between Jul 2015 and Jun 2017 was this for Loyalty. The breakdown of Sales between Jul 2015 and Jun 2017 was this for Household_Type. The breakdown of Sales between Jul 2015 and Jun 2017 was this. Between Jul 2015 and Jun 2017, the total Sales trend was this for Department. Between Jul 2015 and Jun 2017, the total Sales trend was this for Brand. Between Jul 2015 and Jun 2017, the total Sales trend was this for Commodity. Between Jul 2015 and Jun 2017, the total Sales trend was this for Age_Band. Between Jul 2015 and Jun 2017, the total Sales trend was this for Loyalty. Between Jul 2015 and Jun 2017, the total Sales trend was this for Household_Type. Between Jul 2015 and Jun 2017, the total Sales trend was this."

In [0]:
import json

# Create the answers JSON first
answers = {}
answers['text'] = ''
answers['answer_start'] = 0

# Create the qas json second
qas=[]
for i in range(len(input_question)):
  this_qas = {"question": str(input_question[i]),
              "id": str(i),
              "is_impossible": False,
              "answers":[answers]}
  qas.append(this_qas)

# Now create the paragraphs json and add it all to it
paragraphs = {}
paragraphs['qas'] = qas
paragraphs['context'] =  input_context

# Now create the data json
title = 'test'
data = {}
data['title'] = title
data['paragraphs'] = [paragraphs]

# Now finalise it
test_json = {}
test_json['version'] = 'v2.0'
test_json['data'] = [data]

json_test = json.dumps(test_json)

# Write out the json so we can use it properly
with open('json_test.json', 'w') as outfile:
    json.dump(test_json, outfile)


In [0]:
eval_file = 'json_test.json'
eval_examples = read_squad_examples(
        input_file=eval_file, is_training=False,version_2_with_negative=True)



In [42]:
eval_writer = run_squad.FeatureWriter(
        filename=os.path.join(output_dir, "eval.tf_record"),
        is_training=False)
eval_features = []

def append_feature(feature):
  eval_features.append(feature)
  eval_writer.process_feature(feature)

run_squad.convert_examples_to_features(
        examples=eval_examples,
        tokenizer=tokenizer,
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_training=False,
        output_fn=append_feature)
eval_writer.close()

INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 1000000000
INFO:tensorflow:example_index: 0
INFO:tensorflow:doc_span_index: 0
INFO:tensorflow:tokens: [CLS] what is the trend of sales by household _ type [SEP] the top department for quantity between jul 2015 and jun 2017 was this . the top brand for quantity between jul 2015 and jun 2017 was this . the top commodity for quantity between jul 2015 and jun 2017 was this . the top age _ band for quantity between jul 2015 and jun 2017 was this . the top loyalty for quantity between jul 2015 and jun 2017 was this . the top household _ type for quantity between jul 2015 and jun 2017 was this . the top household _ type for quantity between jul 2015 and jun 2017 was this . . the bottom department for quantity between jul 2015 and jun 2017 was this . the bottom brand for quantity between jul 2015 and jun 2017 was this . the bottom commodity for quantity between jul 2015 and jun 2017 was this . the bottom age _ band for quantity between

In [0]:
predict_input_fn = run_squad.input_fn_builder(
        input_file=eval_writer.filename,
        seq_length=max_seq_length,
        is_training=False,
        drop_remainder=False)

In [44]:
all_results = []
for result in estimator.predict(
  predict_input_fn, yield_single_examples=True):
  if len(all_results) % 1000 == 0:
    tf.logging.info("Processing example: %d" % (len(all_results)))
  unique_id = int(result["unique_ids"])
  start_logits = [float(x) for x in result["start_logits"].flat]
  end_logits = [float(x) for x in result["end_logits"].flat]
  all_results.append(
      run_squad.RawResult(
          unique_id=unique_id,
          start_logits=start_logits,
          end_logits=end_logits))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 384)
INFO:tensorflow:  name = input_mask, shape = (?, 384)
INFO:tensorflow:  name = segment_ids, shape = (?, 384)
INFO:tensorflow:  name = unique_ids, shape = (?,)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, 

In [0]:
n_best_size=20
do_lower_case=True
verbose_logging=False
version_2_with_negative=True
null_score_diff_threshold=0.0
all_predictions,all_nbest_json = write_predictions(eval_examples, eval_features, all_results,
                  n_best_size, max_answer_length,
                  do_lower_case)

In [46]:
n=1
print(input_question)
print([x[n] for x in list(all_predictions.items())])

['What is the trend of Sales by Household_Type']
['']


In [47]:
all_nbest_json

OrderedDict([('0',
              [OrderedDict([('text', ''),
                            ('probability', 0.9898992165660715),
                            ('start_logit', 5.0583038330078125),
                            ('end_logit', 5.007004737854004)]),
               OrderedDict([('text',
                             'The breakdown of Sales between Jul 2015 and Jun 2017 was this for Department.'),
                            ('probability', 0.0005855198397399823),
                            ('start_logit', 1.5260491371154785),
                            ('end_logit', 1.1064010858535767)]),
               OrderedDict([('text',
                             'The breakdown of Sales between Jul 2015 and Jun 2017 was this for Department. The breakdown of Sales between Jul 2015 and Jun 2017 was this for Brand.'),
                            ('probability', 0.0005832171602283242),
                            ('start_logit', 1.5260491371154785),
                            ('end_logit', 1.1